In [1]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision.models
from PIL import Image

In [2]:
meta_data = pd.read_csv("/kaggle/input/ham1000-segmentation-and-classification/GroundTruth.csv")

In [3]:
meta_data.head(5)

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# which class does each id belong to?
columns = meta_data.columns[1:]
meta_data["label"] = ""
for i in range(len(meta_data)):
    for col in columns:
        if(int(meta_data.loc[i][col]) == 1):
            meta_data.loc[i,"label"]+=col
    

In [5]:
meta_data.head(5)

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC,label
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL


In [6]:
meta_data["label"].value_counts()

label
NV       6705
MEL      1113
BKL      1099
BCC       514
AKIEC     327
VASC      142
DF        115
Name: count, dtype: int64

## As we can see there is a huge class imbalance

In [7]:
#load the images from the directory and perform necessary transformation
#loading all images at one killed kernel so dividing and then loading the images
from torchvision.models import inception_v3,Inception_V3_Weights

images = []
labels_one_hot = []

path = "/kaggle/input/ham1000-segmentation-and-classification/images"
weights = Inception_V3_Weights.IMAGENET1K_V1
transforms = weights.transforms()

for i in range(len(meta_data)//2):
    img_id = meta_data.loc[i]["image"]
    label = meta_data.loc[i][1:8]  #one hot encodings of each image label
    img_pth = os.path.join(path,img_id + ".jpg")
    img = Image.open(img_pth)
    images.append(transforms(img))
    labels_one_hot.append(torch.tensor(label)) 

/tmp/ipykernel_2658/2375564321.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  labels_one_hot.append(torch.tensor(label))


In [8]:
for i in range(len(meta_data)//2,len(meta_data)):
    img_id = meta_data.loc[i]["image"]
    label = meta_data.loc[i][1:8]  #one hot encodings of each image label
    img_pth = os.path.join(path,img_id + ".jpg")
    img = Image.open(img_pth)
    images.append(transforms(img))
    labels_one_hot.append(torch.tensor(label)) 

images = torch.stack(images)
labels_one_hot = torch.stack(labels_one_hot)

/tmp/ipykernel_2658/3643847215.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  labels_one_hot.append(torch.tensor(label))


In [12]:
print(images.shape)
print(labels_one_hot.shape)

torch.Size([10015, 3, 299, 299])
torch.Size([10015, 7])


In [11]:
# This notebook has used it its RAM quota (28GB)
# So we shall save the images and labels as pytorch tensors and move to new notebook for model building

torch.save(images,"image.pt")
torch.save(labels_one_hot,"labels.pt")